In [29]:
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms as T

In [134]:
from glob import glob

# Create a list 'files' of the 5856 downloaded images saved locally
xray_files = np.array(glob('chestXrays/train/normal/*'))

In [135]:
# Check that file names have been stored correctly
xray_files

array(['chestXrays/train/normal/NORMAL2-IM-0927-0001.jpeg',
       'chestXrays/train/normal/NORMAL2-IM-1056-0001.jpeg',
       'chestXrays/train/normal/IM-0427-0001.jpeg', ...,
       'chestXrays/train/normal/NORMAL2-IM-1011-0001.jpeg',
       'chestXrays/train/normal/NORMAL2-IM-0826-0001.jpeg',
       'chestXrays/train/normal/NORMAL2-IM-0960-0001.jpeg'], dtype='<U54')

In [136]:
# Create a dataframe with file names so we can use 'apply()' later to transform images to tensors
xray_df = pd.DataFrame(data=xray_files, columns=['file_name'])

In [137]:
def get_shape(tensor):
    return tensor.shape[0]

In [138]:
# Ensure that the dataframe is correctly structured
xray_df.head()

,file_name
0,chestXrays/train/normal/NORMAL2-IM-0927-0001.jpeg
1,chestXrays/train/normal/NORMAL2-IM-1056-0001.jpeg
2,chestXrays/train/normal/IM-0427-0001.jpeg
3,chestXrays/train/normal/NORMAL2-IM-1260-0001.jpeg
4,chestXrays/train/normal/IM-0656-0001-0001.jpeg


In [139]:
# Simple function to transform the images for processing
def to_tensor(img_url):
    transform = T.Compose([T.Resize(256), T.Resize((256,364)), T.ToTensor()])
    return transform(Image.open(img_url))

In [140]:
# Verify tensor shape across a few samples
for xray in xray_files[:5]:
    print(to_tensor(xray).shape)
    
to_tensor(xray_files[0]).shape[0]

torch.Size([1, 256, 364])
torch.Size([1, 256, 364])
torch.Size([1, 256, 364])
torch.Size([1, 256, 364])
torch.Size([1, 256, 364])


1

In [141]:
# Create a new column 'tensor' with the to_tensor() function applied to 'file_name'
xray_df['tensor'] = xray_df['file_name'].apply(to_tensor)

In [142]:
def get_shape(tensor):
    return tensor.shape[0]

In [143]:
get_shape(xray_df['tensor'][0])

1

In [144]:
xray_df['channels'] = xray_df['tensor'].apply(get_shape)

In [145]:
xray_df[xray_df['channels']>1]['file_name'].head()

Series([], Name: file_name, dtype: object)

In [146]:
import os

count = 0

for file in xray_df[xray_df['channels']>1]['file_name']:
    os.remove(file)
    count+=1

print('{} files removed.'.format(count))

0 files removed.


In [64]:
# Verify that the dataframe is correctly created
xray_df.head()

,file_name,tensor
0,/Users/supearnesh/Downloads/chest_xray/all/per...,"[[[tensor(0.8157), tensor(0.8078), tensor(0.81..."
1,/Users/supearnesh/Downloads/chest_xray/all/per...,"[[[tensor(0.), tensor(0.), tensor(0.0118), ten..."
2,/Users/supearnesh/Downloads/chest_xray/all/per...,"[[[tensor(0.1765), tensor(0.1765), tensor(0.17..."
3,/Users/supearnesh/Downloads/chest_xray/all/per...,"[[[tensor(0.), tensor(0.), tensor(0.), tensor(..."
4,/Users/supearnesh/Downloads/chest_xray/all/NOR...,"[[[tensor(0.0941), tensor(0.1020), tensor(0.10..."


In [68]:
# Verify that the shape of the tensors are as expected
xray_df['tensor'][1].shape

torch.Size([1, 256, 364])

In [89]:
# Calculate mean and standard deviation of image tensors
tensor_vals = []

for tensor in xray_df['tensor'][:4]:
    tensor_vals.append(tensor.mean())

img_mean = np.mean(tensor_vals)
img_std = np.std(tensor_vals)
    
print('mean: {}'.format(img_mean))
print('std: {}'.format(img_std))

mean: 0.4925113022327423
std: 0.053436361253261566


In [90]:
from PIL import Image
from torchvision import transforms as T

# Looks like multiple labels can be associated with images so we'll have to parse this in the neural network

def pre_process_image(img_url):
    # Load image from file
    img = Image.open(img_url)

    # all pre-trained models expect input images normalized in the same way
    # i.e. mini-batches of 1-channel grayscale images of shape (1 x H x W)
    # H and W are expected to be at least 224
    transform = T.Compose([T.Resize(224), T.CenterCrop(224), T.ToTensor()])
    transformed_img = transform(img)
    
    # the images have to be loaded in to a range of [0, 1]
    # then normalized using mean = [0.4925113] and std = [0.05343636]
    normalize = T.Normalize(mean=img_mean, std=img_std)
    normalized_img = normalize(transformed_img)
    
    # model loading
    # tensor_img = normalized_img.unsqueeze(0)
    
    return normalized_img

In [86]:
xray_df['file_name'][2]

In [87]:
pre_process_image('chestXrays/all/person604_bacteria_2463.jpeg')

0    [[[tensor(-5.1071), tensor(-4.9603), tensor(-5...
1    [[[tensor(-1.7312), tensor(-1.5845), tensor(-1...
2    [[[tensor(-7.1619), tensor(-7.1619), tensor(-7...
3    [[[tensor(-3.8595), tensor(-3.7127), tensor(-3...
4    [[[tensor(-7.1619), tensor(-7.1619), tensor(-7...
Name: normalized_tensor, dtype: object

In [88]:
pre_process_image('chestXrays/all/person604_bacteria_2463.jpeg').shape

torch.Size([1, 224, 224])

In [91]:
import os
from PIL import Image
from torchvision import datasets
from torchvision import transforms as T
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

# Set PIL to be tolerant of image files that are truncated.
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### DONE: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes

transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), T.Normalize(mean=img_mean, std=img_std)])

dataset_train = datasets.ImageFolder('/Users/supearnesh/Downloads/chest_xray/train', transform=transform)
dataset_val = datasets.ImageFolder('/Users/supearnesh/Downloads/chest_xray/val', transform=transform)
dataset_test = datasets.ImageFolder('/Users/supearnesh/Downloads/chest_xray/test', transform=transform)

loader_train = DataLoader(dataset_train, batch_size=1, shuffle=False)
loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False)
loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False)

loaders_cnn = {'train': loader_train, 'valid': loader_val, 'test': loader_test}

In [92]:
# the following import is required for training to be robust to truncated images
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # check if CUDA is available
            use_cuda = torch.cuda.is_available()
            # move to GPU if CUDA is available
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # check if CUDA is available
            use_cuda = torch.cuda.is_available()
            # move tensors to GPU if CUDA is available
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## DONE: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print(f'The validation loss has decreased from {valid_loss_min} to {valid_loss}.')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)
            
    # return trained model
    return model

In [93]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ## Choose an architecture
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        # convolutional layer (sees 1x256x364 tensor)
        self.conv_01 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        # batch normalization applied to convolutional layer
        self.norm_01 = nn.BatchNorm2d(32)
        # convolutional layer (sees 112x112x32 tensor)
        self.conv_02 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        # batch normalization applied to convolutional layer
        self.norm_02 = nn.BatchNorm2d(64)
        # convolutional layer (sees 56x56x64 tensor)
        self.conv_03 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        # batch normalization applied to convolutional layer
        self.norm_03 = nn.BatchNorm2d(128)
        # convolutional layer pooled (sees 28x28x128 tensor)
        self.conv_04 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        # batch normalization applied to convolutional layer
        self.norm_04 = nn.BatchNorm2d(256)
        # convolutional layer pooled (sees 7x7x256 tensor)
        self.conv_05 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        # batch normalization applied to convolutional layer
        self.norm_05 = nn.BatchNorm2d(512)
        # max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # linear layer (7 * 7 * 512 -> 500)
        self.fc_01 = nn.Linear(512 * 7 * 7, 4096)
        # linear layer (4096 -> 133)
        self.fc_02 = nn.Linear(4096, 133)
        # dropout layer (p = 0.50)
        self.dropout = nn.Dropout(0.50)
    
    def forward(self, x):
        ## Define forward behavior
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.norm_01(self.conv_01(x))))
        x = self.pool(F.relu(self.norm_02(self.conv_02(x))))
        x = self.pool(F.relu(self.norm_03(self.conv_03(x))))
        x = self.pool(F.relu(self.norm_04(self.conv_04(x))))
        x = self.pool(F.relu(self.norm_05(self.conv_05(x))))
        # flatten image input
        x = x.view(-1, 7 * 7 * 512)
        # add dropout layer
        x = self.dropout(x)
        # add first hidden layer, with relu activation function
        x = F.relu(self.fc_01(x))
        # add dropout layer
        x = self.dropout(x)
        # add second hidden layer, with relu activation function
        x = self.fc_02(x)
        return x

# create a complete CNN
model_cnn = Net()
print(model_cnn)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move tensors to GPU if CUDA is available
if use_cuda:
    model_cnn.cuda()

Net(
  (conv_01): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_01): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_02): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_02): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_03): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_03): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_04): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_04): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_05): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm_05): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_01): Linear(in_features=25088, out_fe

In [94]:
import torch.optim as optim

# select loss function
criterion_cnn = nn.CrossEntropyLoss()

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move loss function to GPU if CUDA is available
if use_cuda:
    criterion_cnn = criterion_cnn.cuda()

# select optimizer
optimizer_cnn = optim.SGD(model_cnn.parameters(), lr=0.001)

In [95]:
n_epochs = 25

# train the model
model_cnn = train(n_epochs, loaders_cnn, model_cnn, optimizer_cnn, 
                      criterion_cnn, use_cuda, 'model_cnn.pt')

# load the model that got the best validation accuracy
model_cnn.load_state_dict(torch.load('model_cnn.pt'))

Epoch: 1 	Training Loss: 0.005275 	Validation Loss: 7.862362
The validation loss has decreased from inf to 7.862361907958984.
Epoch: 2 	Training Loss: 0.004499 	Validation Loss: 7.202618
The validation loss has decreased from 7.862361907958984 to 7.202617645263672.
Epoch: 3 	Training Loss: 0.003026 	Validation Loss: 7.116066
The validation loss has decreased from 7.202617645263672 to 7.116065979003906.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/supearnesh/Downloads/chest_xray/train/pneumonia/person548_bacteria_2300.jpeg'

In [ ]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [ ]:
test(loaders_cnn, model_cnn, criterion_cnn, use_cuda)

In [ ]:
data_cnn = loaders_cnn

In [ ]:
data_cnn['train'].dataset.classes

In [ ]:
data_cnn = loaders_cnn

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_cnn['train'].dataset.classes]

def predict_xrays(img_path):
    # load the image and return the predicted breed
    global model_cnn
    
    # load image from file
    img = Image.open(img_path)
    
    # all pre-trained models expect input images normalized in the same way
    # i.e. mini-batches of 1-channel grayscale images of shape (1 x H x W)
    # H and W are expected to be at least 224
    
    # the images have to be loaded in to a range of [0, 1]
    # then normalized using mean = [0.60728765] and std = [0.10230779]
    #transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(), T.Normalize(mean=img_mean, std=img_std)])
    
    transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
    transformed_img = transform(img)
    
    print('transformed_img.shape: {}'.format(transformed_img.shape))
    
    # the images have to be loaded in to a range of [0, 1]
    # then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]
    normalize = T.Normalize(mean=img_mean, std=img_std)
    normalized_img = normalize(transformed_img)
    
    print('normalized_img.shape: {}'.format(normalized_img.shape))
    
    tensor_img = tf.image.grayscale_to_rgb(normalized_img.unsqueeze(0))
    
    print('rgb_img.shape: {}'.format(rgb_img.shape))
    
    # model loading
    #tensor_img = normalized_img.unsqueeze(0)
    
    print('tensor_img.shape: {}'.format(tensor_img.shape))
    
    # check if CUDA is available
    use_cuda = torch.cuda.is_available()
    
    # move image tensor to GPU if CUDA is available
    if use_cuda:
        tensor_img = tensor_img.cuda()
    
    # make prediction by passing image tensor to model
    prediction = model_cnn(tensor_img)
    
    # move model prediction to GPU if CUDA is available
    if use_cuda:
        model_cnn = model_cnn.cuda()
    
    # convert predicted probabilities to class index
    tensor_prediction = torch.argmax(prediction)
    
    # move prediction tensor to CPU if CUDA is available
    if use_cuda:
        tensor_prediction = tensor_prediction.cpu()
    
    predicted_class_index = int(np.squeeze(tensor_prediction.numpy()))
    
    return class_names[predicted_class_index] # predicted class index

In [ ]:
xray_files

In [ ]:
test_img = xray_files[10]

predict_xrays(test_img)